# Basic RAG-llama3.1 Example

This notebook shows the following: 
* Explains the typical process of a RAG system
* A query about Sage without RAG
* A query about Sage with RAG and a few tests

---

## Explanation of RAG System

**Setting up RAG**
* Data retrieval and ingestion
  * Chunking
  * Data representation
* Embed data
  * Choice of model
* Store embeddings in vector database
  * Choice of vector base

**LLM-RAG Pipeline**
* Receive prompt
* Embed prompt
* Query vector database for vectors (RETRIEVAL)
  * Similarity process (e.g., cosine, ANN) 
  * Selection of k top passages
  * Optional: advanced techniques like reranking and activate retrieval
* Decode vectors to context
* Feed new context into model with prompt (GENERATION)

> Notes
> * Context is not saved betwen each `ollama.generate()`
> * Site for embedding models: https://huggingface.co/spaces/mteb/leaderboard


## Before RAG

In [3]:
output = ollama.generate(model='llama3.1', prompt='What is the Sage project?')

In [4]:
print(output['response'])

The Sage project is a free and open-source mathematical software system that provides a comprehensive platform for research, education, and development in mathematics. It was started in 1997 by William Stein, a mathematician at Harvard University.

Sage is designed to be a "one-stop shop" for mathematical computations, providing an integrated environment that allows users to work with various mathematical structures, such as numbers (integers, rationals, reals), algebraic objects (groups, rings, fields), and geometric objects (points, curves, surfaces). The system includes a wide range of features and tools, including:

1. **Symbolic Computation**: Sage can perform symbolic manipulations, solving equations, and computing integrals.
2. **Numerical Computation**: It provides numerical computation capabilities using libraries like NumPy and SciPy.
3. **Algebraic Geometry**: Sage has a robust library for algebraic geometry, including tools for working with curves and surfaces.
4. **Combina

## RAG

* This does not include chunking, reranking, or specialized vector databases/embedding models

In [5]:
import ollama
import numpy as np
from numpy.linalg import norm

In [6]:
# takes a list of strings and encodes them using "embed_model" from Ollama
# outputs those vector encodings as numpy array
def embed_data (data, embed_model):
    vector_list = []

    # embedding data
    for sentence in data:
        sent_emb = ollama.embeddings(model=embed_model, prompt=sentence)
        vector_list.append(sent_emb['embedding'])

    # stored vectors
    data_vectors = np.array(vector_list)

    return data_vectors

In [7]:
# takes a prompt and encodes it using "embed_model" from Ollama
# oututs the associated vector
def embed_prompt (user_prompt, embed_model):
    user_emb = ollama.embeddings(model=embed_model, prompt=user_prompt)
    vector_prompt = np.array(user_emb['embedding'])
    return vector_prompt

In [10]:
# calls helper functions to encode the data and prompt
# it then finds the best context within the data with a cosine similarity test
# outputs the index for the best context and its similarity score with the prompt
def find_best_similarity (data, user_prompt, embed_model):
    best_similarity = 0
    best_data_index = 0

    vector_prompt = embed_prompt(user_prompt, embed_model)
    data_vectors = embed_data(data, embed_model)

    # finding the best 
    for i, vec in enumerate(data_vectors):
        similarity = np.dot(vector_prompt,vec)/(norm(vector_prompt)*norm(vec))
        print(f'Similarity score at index {i}: {similarity}')
        
        if similarity > best_similarity:
            best_similarity = similarity
            best_data_index = i

    return best_similarity, best_data_index

#### Test One

In [11]:
data = [
    'The Sage project is an NSF-funded project at Argonne National Laboratory that explores AI at the Edge', 
    'The Philippines is an archipelago, which means it’s made up of a group of islands—7,641 islands, to be exact.',
    'The Windy City nickname has nothing to do with Chicago’s weather, but it comes from the politics'
    ]

user_prompt = 'What is the Sage project?'
embed_model = 'llama3.1'

score, idx = find_best_similarity(data, user_prompt, embed_model)
print (f'Best score: {score}; index: {idx}\n\n')

Similarity score at index 0: 0.29802225530402804
Similarity score at index 1: 0.39987415625480366
Similarity score at index 2: 0.17156026505283603
Best score: 0.39987415625480366; index: 1




In [12]:
output = ollama.generate(
  model="llama3.1",
  prompt=f"Using this data: {data[idx]}. Respond to this prompt: {user_prompt}"
)
print(output['response'])

Unfortunately, I don't have any information about the "Sage project" or its connection to the data you provided. The data mentions that the Philippines is an archipelago with 7,641 islands, but it doesn't mention anything about a project called "Sage".

If you could provide more context or information about the Sage project, I'd be happy to try and help!


#### Test Two -- Tricking llama3.1 

In [14]:
data = [
    'Sage Sage Sage Sage Sage', 
    'The Philippines is an archipelago, which means it’s made up of a group of islands—7,641 islands, to be exact.',
    'What is the Sage project?'
    ]

user_prompt = 'What is the Sage project?'
embed_model = 'llama3.1'


score, idx = find_best_similarity(data, user_prompt, embed_model)
print (f'Score: {score}; index: {idx}\n\n')

Similarity score at index 0: 0.6315181663914815
Similarity score at index 1: 0.4002249339454586
Similarity score at index 2: 1.0000000000000002
Score: 1.0000000000000002; index: 2




In [15]:
output = ollama.generate(
  model="llama3.1",
  prompt=f"Using this data: {data[idx]}. Respond to this prompt: {user_prompt}"
)
print(output['response'])

Unfortunately, you didn't provide any specific data for me to draw from. However, I can still give a general response about the Sage project.

The Sage project is an open-source mathematical software system that aims to create a comprehensive and user-friendly platform for computational mathematics. The project was started in 2005 by William Stein, a mathematician at Harvard University, with the goal of creating a free and open-source alternative to commercial computer algebra systems (CAS) like Mathematica.

Sage is designed to provide an integrated environment for various mathematical tasks, including:

1. Computer algebra: Sage includes a powerful CAS that can perform symbolic and numerical computations.
2. Number theory: Sage has extensive support for number theory, including algebraic number fields and elliptic curves.
3. Algebra: Sage includes tools for working with groups, rings, and other algebraic structures.
4. Geometry: Sage provides functionality for geometry, including pro

#### Test Three -- Using MXBAI for embedding

In [16]:
data = [
    'The Sage project is an NSF-funded project at Argonne National Laboratory that explores AI at the Edge', 
    'The Philippines is an archipelago, which means it’s made up of a group of islands—7,641 islands, to be exact.',
    'The Windy City nickname has nothing to do with Chicago’s weather, but it comes from the politics'
    ]

user_prompt = 'What is the Sage project?'
embed_model = 'mxbai-embed-large'

score, idx = find_best_similarity(data, user_prompt, embed_model)
print (f'Best score: {score}; index: {idx}\n\n')

Similarity score at index 0: 0.7829698088610217
Similarity score at index 1: 0.27547573347012566
Similarity score at index 2: 0.280417930363531
Best score: 0.7829698088610217; index: 0




In [17]:
## With llama3.1
output = ollama.generate(
  model="llama3.1",
  prompt=f"Using this data: {data[idx]}. Respond to this prompt: {user_prompt}"
)
print(output['response'])

Based on the provided information, here's a response:

The Sage project is an initiative funded by the National Science Foundation (NSF) and conducted at Argonne National Laboratory. It specifically focuses on exploring Artificial Intelligence (AI) applications in Edge computing environments. The details about the project's goals, objectives, and outcomes are not provided, but it appears to be a research-oriented effort investigating AI capabilities within Edge settings.


In [18]:
## With llama3
output = ollama.generate(
  model="llama3",
  prompt=f"Using this data: {data[idx]}. Respond to this prompt: {user_prompt}"
)
print(output['response'])

The Sage project is a research initiative funded by the National Science Foundation (NSF) and based at Argonne National Laboratory, which focuses on exploring Artificial Intelligence (AI) in Edge computing environments.


#### Test Four -- Tricking MXBAI

In [19]:
data = [
    'The Sage project is an NSF-funded project at Argonne National Laboratory that explores AI at the Edge', 
    'Sage Sage Sage Sage Sage Sage Sage',
    'What is the Sage project?'
    ]

user_prompt = 'What is the Sage project?'
embed_model = 'mxbai-embed-large'

score, idx = find_best_similarity(data, user_prompt, embed_model)
print (f'Best score: {score}; index: {idx}\n\n')

Similarity score at index 0: 0.7829698088610217
Similarity score at index 1: 0.6319757336654319
Similarity score at index 2: 0.9999999999999998
Best score: 0.9999999999999998; index: 2


